In [3]:
# Author: Shahi
# Purpose: Trading Bot 
# This code defines a machine learning-based trading bot that uses sentiment analysis on news headlines
# to make trading decisions for the SPY stock on the Alpaca trading platform. It implements the MLTrader
# strategy, which determines position sizing, retrieves sentiment from recent news, and executes trades
# based on positive or negative sentiment with high probability. The bot is designed to backtest its strategy
# using historical data from Yahoo Finance.

from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from datetime import datetime 
from alpaca_trade_api import REST 
from timedelta import Timedelta 
from finbert_utils import estimate_sentiment

API_KEY = "PK2F3EVD8Z4BOVVQSIWN"
API_SECRET = "bsBNsZwMIPdivfHiBkgqdV9CGGadoWZnysYfvIm8"
BASE_URL = "https://paper-api.alpaca.markets/v2"

ALPACA_CREDS = {
    "API_KEY":API_KEY, 
    "API_SECRET": API_SECRET, 
    "PAPER": True
}

class MLTrader(Strategy): 
    def initialize(self, symbol:str="SPY", cash_at_risk:float=.5): 
        self.symbol = symbol
        self.sleeptime = "24H" 
        self.last_trade = None 
        self.cash_at_risk = cash_at_risk
        self.api = REST(base_url=BASE_URL, key_id=API_KEY, secret_key=API_SECRET)

    def position_sizing(self): 
        cash = self.get_cash() 
        last_price = self.get_last_price(self.symbol)
        quantity = round(cash * self.cash_at_risk / last_price,0)
        return cash, last_price, quantity

    def get_dates(self): 
        today = self.get_datetime()
        three_days_prior = today - Timedelta(days=3)
        return today.strftime('%Y-%m-%d'), three_days_prior.strftime('%Y-%m-%d')

    def get_sentiment(self): 
        today, three_days_prior = self.get_dates()
        news = self.api.get_news(symbol=self.symbol, 
                                 start=three_days_prior, 
                                 end=today) 
        news = [ev.__dict__["_raw"]["headline"] for ev in news]
        probability, sentiment = estimate_sentiment(news)
        return probability, sentiment 

    def on_trading_iteration(self):
        cash, last_price, quantity = self.position_sizing() 
        probability, sentiment = self.get_sentiment()

        if cash > last_price: 
            if sentiment == "positive" and probability > .999: 
                if self.last_trade == "sell": 
                    self.sell_all() 
                order = self.create_order(
                    self.symbol, 
                    quantity, 
                    "buy", 
                    type="bracket", 
                    take_profit_price=last_price*1.20, 
                    stop_loss_price=last_price*.95
                )
                self.submit_order(order) 
                self.last_trade = "buy"
            elif sentiment == "negative" and probability > .999: 
                if self.last_trade == "buy": 
                    self.sell_all() 
                order = self.create_order(
                    self.symbol, 
                    quantity, 
                    "sell", 
                    type="bracket", 
                    take_profit_price=last_price*.8, 
                    stop_loss_price=last_price*1.05
                )
                self.submit_order(order) 
                self.last_trade = "sell"

start_date = datetime(2020,1,1)
end_date = datetime(2024,7,1) 
broker = Alpaca(ALPACA_CREDS) 
strategy = MLTrader(name='mlstrat', broker=broker, 
                    parameters={"symbol":"SPY", 
                                "cash_at_risk":.5})
strategy.backtest(
    YahooDataBacktesting, 
    start_date, 
    end_date, 
    parameters={"symbol":"SPY", "cash_at_risk":.5}
)
# trader = Trader()
# trader.add_strategy(strategy)
# trader.run_all()

Starting backtest for MLTrader...
Progress ||  14.02%  [Elapsed: 0:00:50 ETA: 0:05:10] Portfolio Val: 181,701.522024-07-23 21:32:58,633: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 168, in _run_forever
    await self._start_ws()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 129, in _start_ws
    await self._auth()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 76, in _auth
    raise ValueError("failed to authenticate")
ValueError: failed to authenticate
Progress ||  14.57%  [Elapsed: 0:00:52 ETA: 0:05:07] Portfolio Val: 187,600.132024-07-23 21:33:00,551: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):


Progress ||  33.25%  [Elapsed: 0:01:53 ETA: 0:03:48] Portfolio Val: 198,551.692024-07-23 21:34:01,899: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 168, in _run_forever
    await self._start_ws()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 129, in _start_ws
    await self._auth()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 76, in _auth
    raise ValueError("failed to authenticate")
ValueError: failed to authenticate
Progress ||  33.98%  [Elapsed: 0:01:55 ETA: 0:03:43] Portfolio Val: 198,551.692024-07-23 21:34:03,416: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdo

Progress ||  52.97%  [Elapsed: 0:02:55 ETA: 0:02:35] Portfolio Val: 335,107.722024-07-23 21:35:03,745: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 168, in _run_forever
    await self._start_ws()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 129, in _start_ws
    await self._auth()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 76, in _auth
    raise ValueError("failed to authenticate")
ValueError: failed to authenticate
Progress ||  53.40%  [Elapsed: 0:02:56 ETA: 0:02:34] Portfolio Val: 272,524.172024-07-23 21:35:04,812: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdo

Progress ||  71.60%  [Elapsed: 0:03:58 ETA: 0:01:34] Portfolio Val: 278,337.852024-07-23 21:36:06,476: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 168, in _run_forever
    await self._start_ws()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 129, in _start_ws
    await self._auth()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 76, in _auth
    raise ValueError("failed to authenticate")
ValueError: failed to authenticate
Progress ||  72.03%  [Elapsed: 0:04:00 ETA: 0:01:33] Portfolio Val: 278,337.852024-07-23 21:36:08,572: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdo

Progress ||  87.67%  [Elapsed: 0:04:57 ETA: 0:00:41] Portfolio Val: 221,004.862024-07-23 21:37:06,002: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 168, in _run_forever
    await self._start_ws()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 129, in _start_ws
    await self._auth()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 76, in _auth
    raise ValueError("failed to authenticate")
ValueError: failed to authenticate
Progress ||  87.79%  [Elapsed: 0:04:58 ETA: 0:00:41] Portfolio Val: 243,602.172024-07-23 21:37:06,946: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdo

...

2024-07-23 22:21:33,891: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 168, in _run_forever
    await self._start_ws()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 129, in _start_ws
    await self._auth()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 76, in _auth
    raise ValueError("failed to authenticate")
ValueError: failed to authenticate
2024-07-23 22:21:34,506: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 168, in _run_forever
    await self._start_ws()
  File "/Users/shahidullahmuslimdo

2024-07-23 22:22:39,124: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 168, in _run_forever
    await self._start_ws()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 129, in _start_ws
    await self._auth()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 76, in _auth
    raise ValueError("failed to authenticate")
ValueError: failed to authenticate
2024-07-23 22:22:41,415: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 168, in _run_forever
    await self._start_ws()
  File "/Users/shahidullahmuslimdo

2024-07-23 22:23:44,030: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 168, in _run_forever
    await self._start_ws()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 129, in _start_ws
    await self._auth()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 76, in _auth
    raise ValueError("failed to authenticate")
ValueError: failed to authenticate
2024-07-23 22:23:45,526: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 168, in _run_forever
    await self._start_ws()
  File "/Users/shahidullahmuslimdo

2024-07-23 22:24:49,851: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 168, in _run_forever
    await self._start_ws()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 129, in _start_ws
    await self._auth()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 76, in _auth
    raise ValueError("failed to authenticate")
ValueError: failed to authenticate
2024-07-23 22:25:36,553: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 168, in _run_forever
    await self._start_ws()
  File "/Users/shahidullahmuslimdo

2024-07-23 22:26:40,259: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 168, in _run_forever
    await self._start_ws()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 129, in _start_ws
    await self._auth()
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 76, in _auth
    raise ValueError("failed to authenticate")
ValueError: failed to authenticate
2024-07-23 22:26:41,295: alpaca.trading.stream: ERROR: error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/shahidullahmuslimdost/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 168, in _run_forever
    await self._start_ws()
  File "/Users/shahidullahmuslimdo